# <center> <img src="../img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Lab 09 - Watermarking with Spark** </center>
---
## <center> **Big Data** </center>
---
### <center> **Spring 2025** </center>
---
### <center> **04/09/2025** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

**Team**: Foraneos

**Students**: Eddie, Konrad 

In [16]:
import findspark
findspark.init()

#### Spark Session creation

In [17]:
from pyspark.sql import SparkSession

konrad_port = "0638c7435d1d"
eddie_port = "8776010e8f6a"

spark = SparkSession.builder \
    .appName("SparkSQLStructuredStreaming-Kafka") \
    .master("spark://{}:7077".format(eddie_port)) \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

### Kafka Stream init

In [18]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", "7f12b05dfecd:9093") \
                .option("subscribe", "kafka-spark-example") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



### Transform binary data into string

In [19]:
kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))

In [20]:
from pyspark.sql.functions import explode, split

words = kafka_df.select(explode(split(kafka_df.value, " ")).alias("word"), "timestamp")
words.printSchema()

root
 |-- word: string (nullable = false)
 |-- timestamp: timestamp (nullable = true)



### Use Watermarking to handle late arrival events

In [21]:
from pyspark.sql.functions import window, avg, min, max
windowed_counts =  words \
                        .withWatermark("timestamp", "2 minutes") \
                        .groupBy(window(words.timestamp, 
                                        "60 seconds", # Window duration 
                                        "30 seconds"), # Slide duration
                                 words.word) \
                        .agg(min("value"), avg("value"), max("value"))
8


AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `value` cannot be resolved. Did you mean one of the following? [`word`, `window`, `timestamp`].;
'Aggregate [window#277, word#265], [window#277 AS window#268, word#265, min('value) AS min(value)#274, avg('value) AS avg(value)#275, max('value) AS max(value)#276]
+- Filter isnotnull(timestamp#246)
   +- Expand [[named_struct(start, knownnullable(precisetimestampconversion(((precisetimestampconversion(timestamp#246, TimestampType, LongType) - CASE WHEN (((precisetimestampconversion(timestamp#246, TimestampType, LongType) - 0) % 30000000) < cast(0 as bigint)) THEN (((precisetimestampconversion(timestamp#246, TimestampType, LongType) - 0) % 30000000) + 30000000) ELSE ((precisetimestampconversion(timestamp#246, TimestampType, LongType) - 0) % 30000000) END) - 0), LongType, TimestampType)), end, knownnullable(precisetimestampconversion((((precisetimestampconversion(timestamp#246, TimestampType, Lon

### Sink configuration

In [ ]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

query = windowed_counts \
                .writeStream \
                .outputMode("update") \
                .trigger(processingTime='30 seconds') \
                .format("console") \
                .option("truncate", "false") \
                .start()

query.awaitTermination(300)

25/04/10 05:40:11 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-de5768eb-2706-47cb-9c5a-90edd10f7702. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/04/10 05:40:11 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/04/10 05:40:11 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+----+-----+
|window|word|count|
+------+----+-----+
+------+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+----+-----+
|window                                    |word|count|
+------------------------------------------+----+-----+
|{2025-04-10 05:40:00, 2025-04-10 05:41:00}|41  |1    |
|{2025-04-10 05:40:00, 2025-04-10 05:41:00}|532 |1    |
|{2025-04-10 05:40:00, 2025-04-10 05:41:00}|63  |3    |
|{2025-04-10 05:40:30, 2025-04-10 05:41:30}|42  |1    |
|{2025-04-10 05:40:00, 2025-04-10 05:41:00}|42  |1    |
|{2025-04-10 05:40:30, 2025-04-10 05:41:30}|532 |1    |
|{2025-04-10 05:40:30, 2025-04-10 05:41:30}|63  |3    |
|{2025-04-10 05:40:30, 2025-04-10 05:41:30}|41  |1    |
+------------------------------------------+----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+------+----+-----+
|window|word|count|
+------+----+-----+
+------+----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+----+-----+
|window                                    |word|count|
+------------------------------------------+----+-----+
|{2025-04-10 05:41:00, 2025-04-10 05:42:00}|6   |1    |
|{2025-04-10 05:41:30, 2025-04-10 05:42:30}|4   |1    |
|{2025-04-10 05:41:30, 2025-04-10 05:42:30}|5   |2    |
|{2025-04-10 05:41:30, 2025-04-10 05:42:30}|53  |1    |
|{2025-04-10 05:41:30, 2025-04-10 05:42:30}|2   |2    |
|{2025-04-10 05:41:00, 2025-04-10 05:42:00}|2   |2    |
|{2025-04-10 05:41:00, 2025-04-10 05:42:00}|23  |2    |
|{2025-04-10 05:41:30, 2025-04-10 05:42:30}|45  |2    |
|{2025-04-10 05:41:30, 2025-04-10 05:42:30}|6   |1    |
|{2025-04-10 05:41:30, 2025-04-10 05:42:30}|43  |1    |
|{2025-04-10 05:41:00, 2025-04-10 05:42:00}|5   |2    |
|{2025-04-10 05:41:00, 2025-04-10 05:42:00}|53  |1    |
|{2025-04-10 05:41:00, 2025-04-10 05:42:00}|45  |2    |
|{2025-

-------------------------------------------
Batch: 4
-------------------------------------------
+------+----+-----+
|window|word|count|
+------+----+-----+
+------+----+-----+



-------------------------------------------
Batch: 5
-------------------------------------------
+------------------------------------------+----+-----+
|window                                    |word|count|
+------------------------------------------+----+-----+
|{2025-04-10 05:42:30, 2025-04-10 05:43:30}|345 |1    |
|{2025-04-10 05:42:30, 2025-04-10 05:43:30}|34  |1    |
|{2025-04-10 05:42:30, 2025-04-10 05:43:30}|35  |1    |
|{2025-04-10 05:43:00, 2025-04-10 05:44:00}|3   |1    |
|{2025-04-10 05:42:30, 2025-04-10 05:43:30}|45  |1    |
|{2025-04-10 05:43:00, 2025-04-10 05:44:00}|35  |1    |
|{2025-04-10 05:42:30, 2025-04-10 05:43:30}|53  |2    |
|{2025-04-10 05:42:30, 2025-04-10 05:43:30}|3   |1    |
|{2025-04-10 05:43:00, 2025-04-10 05:44:00}|345 |1    |
|{2025-04-10 05:43:00, 2025-04-10 05:44:00}|34  |1    |
|{2025-04-10 05:43:00, 2025-04-10 05:44:00}|45  |1    |
|{2025-04-10 05:43:00, 2025-04-10 05:44:00}|53  |2    |
+------------------------------------------+----+-----+



-------------------------------------------
Batch: 6
-------------------------------------------
+------+----+-----+
|window|word|count|
+------+----+-----+
+------+----+-----+



False

In [ ]:
query.stop()
sc.stop()